In [ ]:
#Imports
import pandas as pd
import re
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

from nltk.stem import PorterStemmer
import ast

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from gensim.models import KeyedVectors
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

nltk.download('words')
words = set(nltk.corpus.words.words())

# Set options to display the entire text column
pd.set_option('display.max_colwidth', None)

! pip install langdetect
from langdetect import detect

In [ ]:
!pip install googletrans==4.0.0rc1
from googletrans import Translator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
fb_file_path = '/content/drive/MyDrive/NLP Project/NLP Project/data_nlp/agr_hi_fb_gold.csv'
tw_file_path = '/content/drive/MyDrive/NLP Project/NLP Project/data_nlp/agr_hi_tw_gold.csv'

### Pre-processing facebook data

In [ ]:
df_fb = pd.read_csv(fb_file_path, names=['id', 'text', 'class'])
df_fb = df_fb[['text', 'class']]
df_fb

,text,class
0,Abe movi m bhi dhrm agya kya paglpnti h ye,OAG
1,Badnam Gujat kutte ki olaad madrchod Pakistani kutto suar ki olaad madrchod Pakistani kutto suar ki olaad,OAG
2,Modi ji aap ek bar Up ka CM ko ye afar de fir dekhe Pakistan ki chhathi ka dud yed dila denge,CAG
3,Kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji. Bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir PAk vaalo ka ho jaayega to aap ko sahi hai na Jai hind,CAG
4,"मुसलमान अपना धर्म तो जानते नही लेकिन काँपी कर हिन्दु से सवाल पुछते रहते है जरा हमारे भी सवालो के जवाब दे दो हिम्मत है तो ।\n1) इस्लाम बनाने वाले मोहम्मद ने बुढापे मे अपने साथी अबु-बकर की 9 साल की बच्ची आयशा से बलात्कार किया, वो भी 9 साल की बच्ची से। क्या एैसे बलात्कारी की बनाई बातो पे इमान लाना सही है ?\n2) मुहम्मद ने अपने बेटे जैद की बीबी जैनब से बलात्कार कर शादी क्यु की जैद हीजडा था ? या मुहम्मद आदत से मजबूर था ?\n3) आप बकरा ईद क्यों मनाते हो ? क्योंकि आपके किन्ही साहब ने अल्लाह के आदेश पे अपने बच्चे की क़ुरबानी देनी चाही तब वहा बच्चे की जगह जानवर प्रगट हो गया , तो आप अपने बच्चों की क़ुरबानी दो ना, जानवर की क्यों देते हो अल्लाह सच्चा होगा तो बच्चे की जगह जानवर प्रगट कर देगा और ये अल्लाह है या राक्षस जो खुन खराबे से प्रसंन्न होता है ?\n4) एक तरफ तो मुर्ती पुजा की विरोध करते हो दुसरी तरफ मक्का मे हजरे अस्वाद फत्तर को चुमने जाते हो फत्तर मे विश्वास नही बोलते फिर मीना की जगह फत्तर को शैतान समझ फत्तर क्यु मारते ?\n5) क्यों जब मुसलमान मक्का जाते हो तो सब मुंडन कर हिन्दू जैसे सफ़ेद वस्त्र पहनते हो ? तवाफ की पुरी प्रक्रिया हिन्दु की नकल करते हो फिर मुर्ती पुजा का क्यु विरोध करते हो ?? मजार दरगाहो पे क्या होता है ?\n6) क्यों हिन्दुओँ की तरह उस जगह की परिक्रमा करते हो ? जब अल्लाह सातवे आसमान पे बैठा है तो काबा को क्यु सजदा, नमस्कार करते हो ?\n7) इस्लाम तो बहुत शांति का धर्मं है (आपके अनुसार), तो ऐसा क्यों इस्लाम में लिखा है कि जो काफीर (गैर मुसलमान) है उनको मार दो, आपको 72 हूरें मिलेगी ? भाई अपनी हूरों के चक्कर में हमारी जान क्यों लेना चाहते हो, थोड़ी तो इंसानीयत रखो यार!\n8) कुरान 2:223 मे लिखा है औरतों को खेती की तरह जैसे चाहे इस्तेमाल करो कुरान 33:33 कहती है औरतें घर मे बच्चे पैदा करती रहे और दिन मे 5 बार नमाज फुकती रहे क्या यही इज्जत है औरत की इस्लाम मे ? औरतों को बुर्के मे एैसे कैद कर के रखते हो औरत न हो कोई आटे की बोरी हो ।\n9) क्यों इस्लाम में कहा गया कि पृथ्वी चोकोर है, जबकि हिन्दू धर्मं में पृथ्वी को गोल कहा गया है, तथा साइंस प्रूफ कर चूका है ।\n10) क्यों इस्लाम में कहा गया कि सूरज दलदल में डूब जाता है, जबकी हिन्दू धर्मं में लिखा है कि पृथ्वी सूर्य के चक्कर लगाती है ? साइंस भी प्रूफ कर चूका है हिन्दू धर्मं ने दुनिया को विमान शास्त्र, आयुर्वेद, योग और हजारों चीज़े दी हैं, इस्लाम ने दुनिया को क्या दिया, तबाही और माराकाट के अलावा ?\n11) 1400 साल पहले आपका धर्म स्थापित हुआ था, उससे पहले आप क्या थे ? आपकी इबादत का तरीका क्या था ?\n12) हमारा हिन्दू धर्मं हजारो साल पुराना है हमारा राम सेतु आपने भी देखा होगा, पानी में डूबी श्री कृष्णा की नगरी भी देख ली है, आपके पास क्या सबूत है कि अल्लाह ने मोहम्मद को ही भेजा था ?\n13) मुसलमान जिस भी क्षेत्र मेँ जाते है या दूसरे के या खुद के धर्म वाले के साथ रहते है वहाँ इतनी अशांति क्यो हो जाती है ?\n14) कुरान 2:260 मे अल्लाह इब्राहीम को यकीन दिलाने के लिए कहता है चार जानवर लो फिर उनकी मुन्डीया काटकर एक-एक कर पहाड पर रखो फिर उन्हे आवाज लगाओ देखो मुन्डीया कैसे दौडती हुई आयेगी । सवाल उठता है क्या अल्लाह खुद को सिद्ध करने एैसे फिजूल के कारनामे करता रहता है ? क्या एैसा होना संभव है ? इस्से क्या अल्लाह सिद्ध हो जायेगा ?\n15) अगर इस्लाम सच्चा धर्म है तो अल्लाह मुसलमानो को खतना करके क्यु नही भेजता ? क्यु मासूम बच्चे का खतना कर उसे हिन्दु से मुसलमान बनाते हो ?\n16) काल्पनिक कयामत के दिन कब्र से किसे जिन्दा किया जायेगा हड्डियो को ? क्यु की कब्र मे हड्डियाँ ही बचती है हड्डियाँ जन्नत जाकर क्या हुर्रो के साथ डिस्को करेगी ?",OAG
...,...,...
965,"श्रीमान मोदी साहब,\n आपके द्वारा जो 500 और 1000 के पुराने नोटों को बंद कर दिया जाने का अभूतपूर्व फैसला लिया गया है जिससे की छुपा धन बाहर आ जायेगा किन्तु कई धन्ना सेठो के पास इतना अधिक नगद होगा की वे उसे सार्वजानिक नही कर पाएंगे और स

In [ ]:
# Remove multiple occuring '.' '?' '|' ' '
def reduce_multiple_occurrences(text, symbols):
    # Create a pattern to match consecutive occurrences of specified symbols
    pattern = f'([{re.escape("".join(symbols))}])\\1+'
    # Use re.sub to replace consecutive occurrences with a single occurrence
    cleaned_text = re.sub(pattern, r'\1', text)
    return cleaned_text

In [ ]:
def basic_cleanup(list):
    clean_text_list = []
    for text in list:
      clean_text = text.lower()
      #Remove URLs
      clean_text = re.sub(r'http\S+', '', clean_text)
      #Remove mentions
      clean_text = re.sub(r'@\w+', '', clean_text)
      #Remove hashtags
      clean_text = re.sub(r'#\w+', '', clean_text)
      #Remove \n
      clean_text = repr(clean_text).replace(r'\n', ' ')
      #Remove digits, -, () and other symbols
      clean_text = re.sub(r'[0-9\(\)\-:\']', '', clean_text)
      #Remove hashtags
      clean_text = re.sub(r'#\w+', '', clean_text)
      # # Remove . , । ?
      # clean_text = re.sub(r'[.?।,]', '', clean_text)
      # Define a regular expression pattern to match emojis
      emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols and Pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                               u"\U0001F700-\U0001F77F"  # Alchemical Symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               "]+", flags=re.UNICODE)

      # Remove emojis using the pattern
      clean_text = emoji_pattern.sub(r'', clean_text)
      # Remove multiple occurences of symbols to just 1
      symbols_to_reduce = ['.', '?', '|', ' ', ',']
      # symbols_to_reduce = [' ']
      clean_text = reduce_multiple_occurrences(clean_text, symbols_to_reduce)
      clean_text_list.append(clean_text.strip())
    return clean_text_list

In [ ]:
# clean_comments = preprocess(df_fb['text'])
clean_comments = basic_cleanup(df_fb['text'])

In [ ]:
df_fb['clean text'] = clean_comments

In [ ]:
df_fb.head(10)

,text,class,clean text
0,Abe movi m bhi dhrm agya kya paglpnti h ye,OAG,abe movi m bhi dhrm agya kya paglpnti h ye
1,Badnam Gujat kutte ki olaad madrchod Pakistani kutto suar ki olaad madrchod Pakistani kutto suar ki olaad,OAG,badnam gujat kutte ki olaad madrchod pakistani kutto suar ki olaad madrchod pakistani kutto suar ki olaad
2,Modi ji aap ek bar Up ka CM ko ye afar de fir dekhe Pakistan ki chhathi ka dud yed dila denge,CAG,modi ji aap ek bar up ka cm ko ye afar de fir dekhe pakistan ki chhathi ka dud yed dila denge
3,Kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji. Bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir PAk vaalo ka ho jaayega to aap ko sahi hai na Jai hind,CAG,kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji. bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir pak vaalo ka ho jaayega to aap ko sahi hai na jai hind
4,"मुसलमान अपना धर्म तो जानते नही लेकिन काँपी कर हिन्दु से सवाल पुछते रहते है जरा हमारे भी सवालो के जवाब दे दो हिम्मत है तो ।\n1) इस्लाम बनाने वाले मोहम्मद ने बुढापे मे अपने साथी अबु-बकर की 9 साल की बच्ची आयशा से बलात्कार किया, वो भी 9 साल की बच्ची से। क्या एैसे बलात्कारी की बनाई बातो पे इमान लाना सही है ?\n2) मुहम्मद ने अपने बेटे जैद की बीबी जैनब से बलात्कार कर शादी क्यु की जैद हीजडा था ? या मुहम्मद आदत से मजबूर था ?\n3) आप बकरा ईद क्यों मनाते हो ? क्योंकि आपके किन्ही साहब ने अल्लाह के आदेश पे अपने बच्चे की क़ुरबानी देनी चाही तब वहा बच्चे की जगह जानवर प्रगट हो गया , तो आप अपने बच्चों की क़ुरबानी दो ना, जानवर की क्यों देते हो अल्लाह सच्चा होगा तो बच्चे की जगह जानवर प्रगट कर देगा और ये अल्लाह है या राक्षस जो खुन खराबे से प्रसंन्न होता है ?\n4) एक तरफ तो मुर्ती पुजा की विरोध करते हो दुसरी तरफ मक्का मे हजरे अस्वाद फत्तर को चुमने जाते हो फत्तर मे विश्वास नही बोलते फिर मीना की जगह फत्तर को शैतान समझ फत्तर क्यु मारते ?\n5) क्यों जब मुसलमान मक्का जाते हो तो सब मुंडन कर हिन्दू जैसे सफ़ेद वस्त्र पहनते हो ? तवाफ की पुरी प्रक्रिया हिन्दु की नकल करते हो फिर मुर्ती पुजा का क्यु विरोध करते हो ?? मजार दरगाहो पे क्या होता है ?\n6) क्यों हिन्दुओँ की तरह उस जगह की परिक्रमा करते हो ? जब अल्लाह सातवे आसमान पे बैठा है तो काबा को क्यु सजदा, नमस्कार करते हो ?\n7) इस्लाम तो बहुत शांति का धर्मं है (आपके अनुसार), तो ऐसा क्यों इस्लाम में लिखा है कि जो काफीर (गैर मुसलमान) है उनको मार दो, आपको 72 हूरें मिलेगी ? भाई अपनी हूरों के चक्कर में हमारी जान क्यों लेना चाहते हो, थोड़ी तो इंसानीयत रखो यार!\n8) कुरान 2:223 मे लिखा है औरतों को खेती की तरह जैसे चाहे इस्तेमाल करो कुरान 33:33 कहती है औरतें घर मे बच्चे पैदा करती रहे और दिन मे 5 बार नमाज फुकती रहे क्या यही इज्जत है औरत की इस्लाम मे ? औरतों को बुर्के मे एैसे कैद कर के रखते हो औरत न हो कोई आटे की बोरी हो ।\n9) क्यों इस्लाम में कहा गया कि पृथ्वी चोकोर है, जबकि हिन्दू धर्मं में पृथ्वी को गोल कहा गया है, तथा साइंस प्रूफ कर चूका है ।\n10) क्यों इस्लाम में कहा गया कि सूरज दलदल में डूब जाता है, जबकी हिन्दू धर्मं में लिखा है कि पृथ्वी सूर्य के चक्कर लगाती है ? साइंस भी प्रूफ कर चूका है हिन्दू धर्मं ने दुनिया को विमान शास्त्र, आयुर्वेद, योग और हजारों चीज़े दी हैं, इस्लाम ने दुनिया को क्या दिया, तबाही और माराकाट के अलावा ?\n11) 1400 साल पहले आपका धर्म स्थापित हुआ था, उससे पहले आप क्या थे ? आपकी इबादत का तरीका क्या था ?\n12) हमारा हिन्दू धर्मं हजारो साल पुराना है हमारा राम सेतु आपने भी देखा होगा, पानी में डूबी श्री कृष्णा की नगरी भी देख ली है, आपके पास क्या सबूत है कि अल्लाह ने मोहम्मद को ही भेजा था ?\n13) मुसलमान जिस भी क्षेत्र मेँ जाते है या दूसरे के या खुद के धर्म वाले के साथ रहते है वहाँ इतनी अशांति क्यो हो जाती है ?\n14) कुरान 2:260 मे अल्लाह इब्राहीम को यकीन दिलाने के लिए कहता है चार जानवर लो फिर उनकी मुन्डीया काटकर एक-एक कर पहाड पर रखो फिर उन्हे आवाज लगाओ देखो मुन्डीया कैसे दौडती हुई आयेगी । सवाल उठता है क्या अल्लाह खुद को सिद्ध करने एैसे फिजूल के कारनामे करता रहता है ? क्या एैसा होना संभव है ? इस्से क्या अल्लाह सिद्ध हो जायेगा ?\n15) अगर इस्लाम सच्चा धर्म है तो अल

In [ ]:
# Set the threshold for the length of comment
threshold = 500

# Filter rows based on the length of Column1
df_fb_filtered = df_fb[df_fb['clean text'].apply(lambda x: len(x) <= threshold)]

In [ ]:
df_fb_filtered

,text,class,clean text
0,Abe movi m bhi dhrm agya kya paglpnti h ye,OAG,abe movi m bhi dhrm agya kya paglpnti h ye
1,Badnam Gujat kutte ki olaad madrchod Pakistani kutto suar ki olaad madrchod Pakistani kutto suar ki olaad,OAG,badnam gujat kutte ki olaad madrchod pakistani kutto suar ki olaad madrchod pakistani kutto suar ki olaad
2,Modi ji aap ek bar Up ka CM ko ye afar de fir dekhe Pakistan ki chhathi ka dud yed dila denge,CAG,modi ji aap ek bar up ka cm ko ye afar de fir dekhe pakistan ki chhathi ka dud yed dila denge
3,Kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji. Bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir PAk vaalo ka ho jaayega to aap ko sahi hai na Jai hind,CAG,kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji. bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir pak vaalo ka ho jaayega to aap ko sahi hai na jai hind
5,Hmare desh k jawano ko bol diya jye .....bs apne aap dekh lenge kis trh se aisi taisi krni h ......or netao ko narjrband kr diya full powar army or paramilitary ko de di jye ......,CAG,hmare desh k jawano ko bol diya jye .bs apne aap dekh lenge kis trh se aisi taisi krni h .or netao ko narjrband kr diya full powar army or paramilitary ko de di jye .
...,...,...,...
963,Ap B Lagao Apny naam ka Bot 💙 Powered By 💙 Md Adnan 💙 \n\nSelf Site <3 Faxt.tk <3\nSite By 💚 Usama Ahmad 💚,NAG,ap b lagao apny naam ka bot powered by md adnan self site < faxt.tk < site by usama ahmad
964,Inquilab Zindabad Hindustan Zindabad Vande Matram Jai Jawan Jai Kisan,NAG,inquilab zindabad hindustan zindabad vande matram jai jawan jai kisan
967,"Apki khushi ""Ganesh_ji k kaan jaisi badi ho\nGum unke aankh jaisa chhota ho\nJeevan unke soond jaisa lamba\nOr\nAapka Har pal unke laddu jaisa meetha ho\n\nHAPPY GANESH CHATURTHI",NAG,"apki khushi ""ganesh_ji k kaan jaisi badi ho gum unke aankh jaisa chhota ho jeevan unke soond jaisa lamba or aapka har pal unke laddu jaisa meetha ho happy ganesh chaturthi"
968,bangladesh ab wo team nahi rahi jo pahle thi ab wo bhi final mai pahunch sakti hai,NAG,bangladesh ab wo team nahi rahi jo pahle thi ab wo bhi final mai pahunch sakti hai


### Pre-processing twitter data

In [ ]:
df_tw = pd.read_csv(tw_file_path, names=['id', 'text', 'class'])
df_tw = df_tw[['text', 'class']]
df_tw

,text,class
0,Tera baap bambani h .. bambani .. ambani nhi #IndvsSA,CAG
1,Aaj wicket jane per khushi kaam ho hai.... #INDvsUAE,NAG
2,चौका मारकर इतना उड़ रहे हो 😳 सालों छक्का मारा तब तो नंगा नाच करने लग जाओगे BC तुम मैदान में 😂 #IndVsPak,OAG
3,RT @ErVishalInsan: #BlockBusterMSG #MustWatchMSG देखो MSG The Messenger है कमाल\n हर एक की जिंदगी कर देगी खुशियो से माला मालामाल,NAG
4,शिट #IndvsPak,CAG
...,...,...
1189,कुछ तो #BeefBan से इतने दुखी है जैसे साले तीनो टाइम beef ही खाते थे।\nपेटा के लिए नंगे होकर कुत्ते उठा लेंगे पर #BeefBan से अधिकार हनन हो गए,OAG
1190,@deepakkakran एक ये भी है खानदानी तोतला साला @FarOutAkhtar .. पहले अपनी जुबान संभाल जीभ से लाचार आदमी. फिर बोलियो #BeefBan के खिलाफ..,OAG
1191,"Swatch Bharat Abhiyaan ki shurvaat aaj Dilli se hui hai! Bharat vasiyon se nivedan hai, kripaya jhaadu chalaye! #AAPSweep @AamAadmiParty",OAG
1192,RT @iamkrishnam: Kya kya Ban karthe miyan ye bhakt loga? Charminaar ka dum biryani nakko Ban karon miyan. #BeefBan,OAG


In [ ]:
clean_tweets = basic_cleanup(df_tw['text'])

In [ ]:
df_tw['clean text'] = clean_tweets

In [ ]:
df_tw.head(5)

,text,class,clean text
0,Tera baap bambani h .. bambani .. ambani nhi #IndvsSA,CAG,tera baap bambani h . bambani . ambani nhi
1,Aaj wicket jane per khushi kaam ho hai.... #INDvsUAE,NAG,aaj wicket jane per khushi kaam ho hai.
2,चौका मारकर इतना उड़ रहे हो 😳 सालों छक्का मारा तब तो नंगा नाच करने लग जाओगे BC तुम मैदान में 😂 #IndVsPak,OAG,चौका मारकर इतना उड़ रहे हो सालों छक्का मारा तब तो नंगा नाच करने लग जाओगे bc तुम मैदान में
3,RT @ErVishalInsan: #BlockBusterMSG #MustWatchMSG देखो MSG The Messenger है कमाल\n हर एक की जिंदगी कर देगी खुशियो से माला मालामाल,NAG,rt देखो msg the messenger है कमाल हर एक की जिंदगी कर देगी खुशियो से माला मालामाल
4,शिट #IndvsPak,CAG,शिट


In [ ]:
# @title
# from nltk.tokenize import  sent_tokenize

# string = "hm to bolte hai pak kashmirkashmir kr raha hai to,bharat ko pak v chhin lena chaheye aur bharat bna dena chaheye,waise v bharat me jamin ki kami ho rahi rahi? न भक ग कतत,न हग नद खरब|"
# string2 = "सलम बनन वल महममद न बढप म अपन सथ अबबकर क सल क बचच आयश स बलतकर कय, व भ सल क बचच स। कय एस बलतकर क बनई बत प इमन लन सह ह । महममद न अपन बट जद क बब जनब स बलतकर कर शद कय क जद हजड थ ? य महममद आदत स मजबर थ ? आप बकर ईद कय मनत ह ? कयक आपक कनह सहब न अललह क आदश प अपन बचच क करबन दन चह तब वह बचच क जगह जनवर परगट ह गय , त आप अपन बचच क करबन द न, जनवर क कय दत ह अललह सचच हग त बचच क जगह जनवर परगट कर दग और य अललह ह य रकषस ज खन खरब स परसनन हत ह ?"
# string3 = "ýah aurat ke roope pagal aur naggi kutti hai. beshram kahi ki."
# sent_tokenize(string3)

In [ ]:
# @title
# from nltk.tokenize import  sent_tokenize
# clean_comments_sentence_tokens = []
# string2 = "सलम बनन वल महममद न बढप म अपन सथ अबबकर क सल क बचच आयश स बलतकर कय, व भ सल क बचच स। कय एस बलतकर क बनई बत प इमन लन सह ह ? महममद न अपन बट जद क बब जनब स बलतकर कर शद कय क जद हजड थ ? य महममद आदत स मजबर थ ? आप बकर ईद कय मनत ह ? कयक आपक कनह सहब न अललह क आदश प अपन बचच क करबन दन चह तब वह बचच क जगह जनवर परगट ह गय , त आप अपन बचच क करबन द न, जनवर क कय दत ह अललह सचच हग त बचच क जगह जनवर परगट कर दग और य अललह ह य रकषस ज खन खरब स परसनन हत ह ?"
# clean_comments = [string, string2]
# for comment in clean_comments:
#   tokens = sent_tokenize(comment)
#   final_tokens = []
#   for token in tokens:
#     print(token)
#     hindi_sentences = token.split('।')
#     print(len(hindi_sentences))
#     if len(hindi_sentences) == 1:
#       final_tokens.append(token)
#     else:
#       final_tokens.append(hindi_sentences)
#     print("Final = ",final_tokens)
#   clean_comments_sentence_tokens.append(final_tokens)

In [ ]:
# @title
# # @title
# # example of sentence splitter
# strings = ["A dot sentence.A question? a hindi sentence।", "A hindi sentence।A question? beshram kahi ki.। modi ji aap ek bar up ka cm ko ye afar de fir dekhe pakistan ki chhathi ka dud yed dila denge?"]
# splitted_text = ultimate_sentence_splitter(strings)
# splitted_text

In [ ]:
# @title
# Testing the method
# translate(splitted_text[1])

In [ ]:
# @title
# translator = Translator()
# text = df_fb['text'][4]
# text = re.sub('\n', '', text)
# print(text)
# string2 = "बलतकर"
# translated_text = translator.translate(string2, src="hi", dest='en')

# translated_text.text

### Translation & Transliteration

In [ ]:
# This method with split the sntences by . ? and ।
# This way needn't use separate sent_tokenizers for hindi and english
# Also found the sent_tokenizers have certain limitations
def ultimate_sentence_splitter(strings):
  sentences = []
  for string in strings:
    # print("string ",string)
    linesplit = []
    results1 = string.split(".")
    # print(results1)
    for result1 in results1:
      result1 = result1.strip()
      if len(result1) > 1:
        results2 = result1.split('?')
        # print(results2)
        for result2 in results2:
          result2 = result2.strip()
          if len(result2) > 1:
            results3 = result2.split('।')
            # print(results3)
            for result3 in results3:
              result3 = result3.strip()
              if len(result3) > 1:
                # print("here ",result3)
                linesplit.append(result3)
    sentences.append(linesplit)
  return sentences

In [ ]:
import time
# This method will pick up different lines, translate one by one and form a new line translated in english
def translate(lines, translator):
    translated_lines = []
    for line in lines:
      if line is not None and isinstance(line, str):
        translated_text = translator.translate(line, src="hi", dest='en')
        translated_lines.append(translated_text.text)
        time.sleep(1)
      else:
        translated_lines.append(line)
    translated_result = ' '.join(translated_lines)
    return translated_result

#### Facebook comments

In [ ]:
# Testing the methods
splitted_text = ultimate_sentence_splitter(df_fb_filtered['clean text'][0:4])
splitted_text

[['abe movi m bhi dhrm agya kya paglpnti h ye'],
 ['badnam gujat kutte ki olaad madrchod pakistani kutto suar ki olaad madrchod pakistani kutto suar ki olaad'],
 ['modi ji aap ek bar up ka cm ko ye afar de fir dekhe pakistan ki chhathi ka dud yed dila denge'],
 ['kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji',
  'bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir pak vaalo ka ho jaayega to aap ko sahi hai na jai hind']]

In [ ]:
# Testing methods
for text in splitted_text:
  print(translate(text, Translator()))

Abe Movi also, what is the insanity of religion
Badnaam Gaat Dog's Aulad Madarchod Pakistani Dog Pigs Halad Madarachod Pakistani Dog Pig
Modi ji, you will give this offer to the cum once, then see that you will find the edge of Pakistan's Chhathi.
What happens, when someone becomes a Pand, then it remains so much pressure that he knows something will happen Just tell you and support people, tell people that you will get educated yourself, what will be done by the people of Kashmir, then you are right or not Jai Hind


In [ ]:
splitted_fb_text = ultimate_sentence_splitter(df_fb_filtered['clean text'])
splitted_fb_text[0:10]

[['abe movi m bhi dhrm agya kya paglpnti h ye'],
 ['badnam gujat kutte ki olaad madrchod pakistani kutto suar ki olaad madrchod pakistani kutto suar ki olaad'],
 ['modi ji aap ek bar up ka cm ko ye afar de fir dekhe pakistan ki chhathi ka dud yed dila denge'],
 ['kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji',
  'bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir pak vaalo ka ho jaayega to aap ko sahi hai na jai hind'],
 ['hmare desh k jawano ko bol diya jye',
  'bs apne aap dekh lenge kis trh se aisi taisi krni h',
  'or netao ko narjrband kr diya full powar army or paramilitary ko de di jye'],
 ['bjp wale bas muslmano ko target krna jante hain talak',
  'azan etf',
  'muddo ko lekar aur jo jawan shaheed ho rhe hain unke bare me bolna v nhi chahte hai'],
 ['modi aur rajnath ke taraf se mai kathor ,ghor ninda karta hu,'],
 ['bar bar marane se accha hai ek bar hi mare',
  'a

In [ ]:
len(splitted_fb_text)

910

In [ ]:
len(translated_fb_comments)

326

In [ ]:
len(translated_fb_comments_2)

271

In [ ]:
len(translated_fb_comments_3)

307

In [ ]:
translated_fb_comments_4 = []
translator = Translator()
for text in splitted_fb_text[904:910]:
  print(text)
  translated_fb_comments_4.append(translate(text, translator))

['जय भारत', 'आपको व आपके सभी शुभचिंतकों को और सभी परिजनों को वें स्वतंत्रता दिवस की पूर्व संध्या पर हार्दिक बधाई एवं शुभकामनायें']
['ap b lagao apny naam ka bot powered by md adnan self site < faxt', 'tk < site by usama ahmad']
['inquilab zindabad hindustan zindabad vande matram jai jawan jai kisan']
['apki khushi "ganesh_ji k kaan jaisi badi ho gum unke aankh jaisa chhota ho jeevan unke soond jaisa lamba or aapka har pal unke laddu jaisa meetha ho happy ganesh chaturthi']
['bangladesh ab wo team nahi rahi jo pahle thi ab wo bhi final mai pahunch sakti hai']
['sahi baat kahi dheeraj dada aaj aisa mehsoos hua ki koi sacche deshbhakt bache hain']


In [ ]:
result_list_fb = []
result_list_fb.extend(translated_fb_comments)
result_list_fb.extend(translated_fb_comments_2)
result_list_fb.extend(translated_fb_comments_3)
result_list_fb.extend(translated_fb_comments_4)
len(result_list_fb)

910

In [ ]:
import csv

import os
os.chdir("/content/drive/MyDrive/NLP Project/NLP Project/processed_data")

# Reshape the list to a list of lists
data_list_of_lists = [[item] for item in result_list_fb]

# Specify the file name
csv_file_name = "translated_fb_new.csv"

# Writing to CSV file
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_list_of_lists)

print(f"The list has been written to {csv_file_name}.")

The list has been written to translated_fb_new.csv.


In [ ]:
df_fb_filtered['translated text'] = result_list_fb

<ipython-input-50-962b424cbb06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fb_filtered['translated text'] = result_list_fb


In [ ]:
import os
os.chdir("/content/drive/MyDrive/NLP Project/NLP Project/processed_data")
df_fb_filtered.to_csv('fb_comments_clean_translated_new.csv', index=False)

In [ ]:
df_fb_filtered.head(10)

,text,class,clean text,translated text
0,Abe movi m bhi dhrm agya kya paglpnti h ye,OAG,abe movi m bhi dhrm agya kya paglpnti h ye,"Abe Movi also, what is the insanity of religion"
1,Badnam Gujat kutte ki olaad madrchod Pakistani kutto suar ki olaad madrchod Pakistani kutto suar ki olaad,OAG,badnam gujat kutte ki olaad madrchod pakistani kutto suar ki olaad madrchod pakistani kutto suar ki olaad,Badnaam Gaat Dog's Aulad Madarchod Pakistani Dog Pigs Halad Madarachod Pakistani Dog Pig
2,Modi ji aap ek bar Up ka CM ko ye afar de fir dekhe Pakistan ki chhathi ka dud yed dila denge,CAG,modi ji aap ek bar up ka cm ko ye afar de fir dekhe pakistan ki chhathi ka dud yed dila denge,"Modi ji, you will give this offer to the cum once, then see that you will find the edge of Pakistan's Chhathi."
3,Kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji. Bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir PAk vaalo ka ho jaayega to aap ko sahi hai na Jai hind,CAG,kya hota hai ji jb koi pm bn jaata hai to itna pressure rahata hai ki vahi jaanta hai kuch na kuch hoga iss vaar ji. bs aap log aur support kare logo ko bataye aap khud padhe likhe lg rahe hai kya milega kashmir pak vaalo ka ho jaayega to aap ko sahi hai na jai hind,"What happens, when someone becomes a Pand, then it remains so much pressure that he knows something will happen Just tell you and support people, tell people that you will get educated yourself, what will be done by the people of Kashmir, then you are right or not Jai Hind"
5,Hmare desh k jawano ko bol diya jye .....bs apne aap dekh lenge kis trh se aisi taisi krni h ......or netao ko narjrband kr diya full powar army or paramilitary ko de di jye ......,CAG,hmare desh k jawano ko bol diya jye .bs apne aap dekh lenge kis trh se aisi taisi krni h .or netao ko narjrband kr diya full powar army or paramilitary ko de di jye .,Speak to the soldiers of our country Just see how you have to do this And the leaders were put on house arrest to give the full power army and paramilitary
6,Bjp wale bas muslmano ko target krna jante hain talak.azan etf.muddo ko lekar aur jo jawan shaheed ho rhe hain unke bare me bolna v nhi chahte hai,CAG,bjp wale bas muslmano ko target krna jante hain talak.azan etf.muddo ko lekar aur jo jawan shaheed ho rhe hain unke bare me bolna v nhi chahte hai,BJP people know how to target Muslims azan etf Do not want to speak about issues and the soldiers who are being martyred
7,"Modi aur rajnath ke taraf se mai kathor ,ghor ninda karta hu,,😜😜😜😜😜",CAG,"modi aur rajnath ke taraf se mai kathor ,ghor ninda karta hu,","On behalf of Modi and Rajnath, I condemn hard, strong,"
8,Bar bar marane se accha hai ek bar hi mare. Aur apane desh me. Pakistan hi aantkwad badata hai. Pakistan nhi to aantkwad nhi hoga. Usaki punchh kutta jaisa hai. Usako jad se nikal na padega. Nhi to vo aise kayar ki tarah pichhe se hamla karta rahega aur hamara sainik sahid hote rahenge. Jai hind,CAG,bar bar marane se accha hai ek bar hi mare. aur apane desh me. pakistan hi aantkwad badata hai. pakistan nhi to aantkwad nhi hoga. usaki punchh kutta jaisa hai. usako jad se nikal na padega. nhi to vo aise kayar ki tarah pichhe se hamla karta rahega aur hamara sainik sahid hote rahenge. jai hind,"It is better to die again and again And in our country Pakistan only increases terrorism There will be no terrorism if not Pakistan His tail is like a dog He will not get out of the root Otherwise, he will continue to attack from behind like such a coward and our soldiers will continue to be martyred. Jai Hind"
9,Tumhare ghar k koi mre n tb akl ayegi salo,OAG,tumhare ghar k koi mre n tb akl ayegi salo,No one in your house will come and then many years
10,Ýah aurat ke roope pagal aur naggi kutti hai.beshram kahi ki.,OAG,ýah aurat ke roope pagal aur naggi kutti hai.beshram kahi ki.,ýah aurat ke roope pagal aur naggi kutti hai Shameless


#### Twitter tweets

In [ ]:
splitted_tw_text = ultimate_sentence_splitter(df_tw['clean text'])
splitted_tw_text[0:10]

[['tera baap bambani h', 'bambani', 'ambani nhi'],
 ['aaj wicket jane per khushi kaam ho hai'],
 ['चौका मारकर इतना उड़ रहे हो सालों छक्का मारा तब तो नंगा नाच करने लग जाओगे bc तुम मैदान में'],
 ['rt देखो msg the messenger है कमाल हर एक की जिंदगी कर देगी खुशियो से माला मालामाल'],
 ['शिट'],
 ['छठी बार छठी का दूध पाक को याद दिला देना team india'],
 ['fatni shuru pak ki', 'jai hind'],
 ['अरे निर्दयी विजेताओं,जीत हार तो ठीक है,लेकिन क्या किसी ने पड़ोसिओं पाकिस्तान से पुछा,कि उन्होंने खाना खाया भी है या नहीं'],
 ['इस नई सरकार को भुखमरो से निबटना है', 'केजरी की चड्डी बी उतार लेंगे ये'],
 ['shahzaad out!!!!!!!!!!!!! thoko taaaliiii']]

In [ ]:
translated_tw_tweets = []
translator = Translator()
for text in splitted_tw_text:
  translated_tw_tweets.append(translate(text, translator))

In [ ]:
len(translated_tw_tweets)

1194

In [ ]:
translated_tw_tweets[0:10]

['Your father is Bambani bambani Not Ambani',
 'Today, there is a happy job when the wicket is going',
 'You are flying so much for years by hitting a four, then you will start dancing in the field',
 'RT Watch MSG The Messenger is amazing everyone will do the life of happiness',
 'Shit',
 'TEAM India reminds Pakistan for sixth time',
 'Burst starts Jai Hind',
 'Hey ruthless winners, victory defeat is fine, but has anyone asked neighborhood Pakistan, whether they have eaten or not',
 "This new government has to deal with starvation Kejri's tights will be removed",
 'Shahzaad Out !!!!!!!!!!!!!!thoko taaaliiii']

In [ ]:
df_tw['translated text'] = translated_tw_tweets

In [ ]:
import csv

import os
os.chdir("/content/drive/MyDrive/NLP Project/NLP Project/processed_data")

# Reshape the list to a list of lists
data_list_of_lists = [[item] for item in translated_tw_tweets]

# Specify the file name
csv_file_name = "translated_tw_new.csv"

# Writing to CSV file
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_list_of_lists)

print(f"The list has been written to {csv_file_name}.")

The list has been written to translated_tw_new.csv.


In [ ]:
df_tw.to_csv('tw_tweets_clean_translated_new.csv', index=False)

In [ ]:
df_tw.head(10)

,text,class,clean text,translated text
0,Tera baap bambani h .. bambani .. ambani nhi #IndvsSA,CAG,tera baap bambani h . bambani . ambani nhi,Your father is Bambani bambani Not Ambani
1,Aaj wicket jane per khushi kaam ho hai.... #INDvsUAE,NAG,aaj wicket jane per khushi kaam ho hai.,"Today, there is a happy job when the wicket is going"
2,चौका मारकर इतना उड़ रहे हो 😳 सालों छक्का मारा तब तो नंगा नाच करने लग जाओगे BC तुम मैदान में 😂 #IndVsPak,OAG,चौका मारकर इतना उड़ रहे हो सालों छक्का मारा तब तो नंगा नाच करने लग जाओगे bc तुम मैदान में,"You are flying so much for years by hitting a four, then you will start dancing in the field"
3,RT @ErVishalInsan: #BlockBusterMSG #MustWatchMSG देखो MSG The Messenger है कमाल\n हर एक की जिंदगी कर देगी खुशियो से माला मालामाल,NAG,rt देखो msg the messenger है कमाल हर एक की जिंदगी कर देगी खुशियो से माला मालामाल,RT Watch MSG The Messenger is amazing everyone will do the life of happiness
4,शिट #IndvsPak,CAG,शिट,Shit
5,छठी बार छठी का दूध पाक को याद दिला देना Team India #IndWins #IndvsPak,OAG,छठी बार छठी का दूध पाक को याद दिला देना team india,TEAM India reminds Pakistan for sixth time
6,#INDvsPAK\nFatni shuru pak ki...jai hind,OAG,fatni shuru pak ki.jai hind,Burst starts Jai Hind
7,"@aajtak अरे निर्दयी विजेताओं,जीत हार तो ठीक है,लेकिन क्या किसी ने पड़ोसिओं (पाकिस्तान) से पुछा,कि उन्होंने खाना खाया भी है या नहीं?#IndvsPak",CAG,"अरे निर्दयी विजेताओं,जीत हार तो ठीक है,लेकिन क्या किसी ने पड़ोसिओं पाकिस्तान से पुछा,कि उन्होंने खाना खाया भी है या नहीं?","Hey ruthless winners, victory defeat is fine, but has anyone asked neighborhood Pakistan, whether they have eaten or not"
8,इस नई सरकार को भुखमरो से निबटना है। केजरी की चड्डी बी उतार लेंगे ये। #KiskiDilli #AAPSweep #AAP,CAG,इस नई सरकार को भुखमरो से निबटना है। केजरी की चड्डी बी उतार लेंगे ये।,This new government has to deal with starvation Kejri's tights will be removed
9,#IndvsPak Shahzaad OUT!!!!!!!!!!!!! Thoko TAAALIIII... #Indianeedwickets,CAG,shahzaad out!!!!!!!!!!!!! thoko taaaliiii.,Shahzaad Out !!!!!!!!!!!!!!thoko taaaliiii
